In [1]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-32GB (UUID: GPU-8e7bff07-76c8-52a0-2680-c1419e14718d)


In [2]:
!nvidia-smi

Tue Dec  9 22:06:59 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:86:00.0 Off |                    0 |
| N/A   36C    P0              44W / 300W |      0MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
%%writefile single_gpu_amp_benchmark.py
import os
import time
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.multiprocessing as mp
import torch.distributed as dist

from torch.nn import Module
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.distributed import DistributedSampler
from torchvision import transforms
from sklearn.model_selection import train_test_split

from torch.distributed.fsdp import FullyShardedDataParallel as FSDP, ShardingStrategy

# AMP Imports
try:
    from torch.amp import autocast, GradScaler
    USE_NEW_AMP = True
except ImportError:
    from torch.cuda.amp import autocast, GradScaler
    USE_NEW_AMP = False

# ============================================================
# 1. SETUP / CLEANUP
# ============================================================

def setup_dist(rank, world_size):
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "12356" 
    dist.init_process_group(backend="nccl", rank=rank, world_size=world_size)
    torch.cuda.set_device(rank)

def cleanup_dist():
    dist.destroy_process_group()

# ============================================================
# 2. DATASET & MODEL
# ============================================================

class OralCancerDataset(Dataset):
    def __init__(self, dataframe, path_map, transform=None):
        self.data = dataframe.reset_index(drop=True)
        self.path_map = path_map
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_id = row["id"]
        label = int(row["label"])
        
        img_path = self.path_map.get(img_id)
        if img_path is None:
            image = Image.new("RGB", (96, 96))
        else:
            image = Image.open(img_path).convert("RGB")
            
        if self.transform:
            image = self.transform(image)
            
        return image, label

class SimpleCNN(Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 12 * 12, 256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 128 * 12 * 12)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# ============================================================
# 3. TRAINING WORKER (1 GPU + AMP + RESUME)
# ============================================================

def train_worker(rank, world_size, df, path_map, batch_size, num_epochs, csv_path):
    setup_dist(rank, world_size)
    device = torch.device(f"cuda:{rank}")
    
    # Transforms
    IMG_SIZE = 96
    train_tf = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3),
    ])
    val_tf = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3),
    ])

    # Split
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])
    
    train_dataset = OralCancerDataset(train_df, path_map, transform=train_tf)
    val_dataset   = OralCancerDataset(val_df,   path_map, transform=val_tf)

    # Samplers
    train_sampler = DistributedSampler(train_dataset, num_replicas=world_size, rank=rank, shuffle=True)
    val_sampler   = DistributedSampler(val_dataset, num_replicas=world_size, rank=rank, shuffle=False)

    # Loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler, 
                              num_workers=0, pin_memory=True)
    val_loader   = DataLoader(val_dataset, batch_size=batch_size, sampler=val_sampler, 
                              num_workers=0, pin_memory=True)

    # Model Setup
    model = SimpleCNN().to(device)
    sharding_strategy = ShardingStrategy.NO_SHARD
    model = FSDP(model, device_id=device, sharding_strategy=sharding_strategy)

    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    scaler = GradScaler()

    # --- CHECKPOINT & RESUME LOGIC ---
    ckpt_dir = "checkpoints_amp_fresh"
    os.makedirs(ckpt_dir, exist_ok=True)
    ckpt_path = os.path.join(ckpt_dir, f"1gpu_amp_batch{batch_size}.pt")

    start_epoch = 0
    if os.path.exists(ckpt_path):
        print(f"[RESUME] Found checkpoint for Batch {batch_size}. Loading...")
        # Map location is crucial to ensure it loads to correct GPU
        checkpoint = torch.load(ckpt_path, map_location=device)
        model.load_state_dict(checkpoint['model_state'])
        optimizer.load_state_dict(checkpoint['optimizer_state'])
        scaler.load_state_dict(checkpoint['scaler_state'])
        start_epoch = checkpoint['epoch']
        print(f"[RESUME] Resuming from Epoch {start_epoch}")
    else:
        print(f"[START] No checkpoint found. Starting fresh for Batch {batch_size}")

    # Start loop from start_epoch
    for epoch in range(start_epoch, num_epochs):
        train_sampler.set_epoch(epoch)
        torch.cuda.reset_peak_memory_stats(device)
        epoch_start = time.time()

        # --- TRAIN ---
        model.train()
        train_loss_sum = 0.0
        train_correct = 0
        train_total = 0

        for images, labels in train_loader:
            images = images.to(device, non_blocking=True)
            labels = labels.float().unsqueeze(1).to(device, non_blocking=True)
            
            optimizer.zero_grad()
            
            if USE_NEW_AMP:
                amp_ctx = autocast(device_type="cuda")
            else:
                amp_ctx = autocast()
                
            with amp_ctx:
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss_sum += loss.item() * images.size(0)
            probs = torch.sigmoid(outputs)
            preds = (probs >= 0.5).float()
            train_correct += (preds == labels).sum().item()
            train_total += labels.size(0)

        epoch_time = time.time() - epoch_start

        # --- VALIDATION ---
        model.eval()
        val_loss_sum = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device, non_blocking=True)
                labels = labels.float().unsqueeze(1).to(device, non_blocking=True)
                
                if USE_NEW_AMP:
                    amp_ctx = autocast(device_type="cuda")
                else:
                    amp_ctx = autocast()
                
                with amp_ctx:
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                
                val_loss_sum += loss.item() * images.size(0)
                probs = torch.sigmoid(outputs)
                preds = (probs >= 0.5).float()
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        # Metrics
        avg_train_loss = train_loss_sum / train_total
        avg_train_acc  = train_correct / train_total
        avg_val_loss   = val_loss_sum / val_total
        avg_val_acc    = val_correct / val_total
        
        peak_mem = torch.cuda.max_memory_allocated(device)
        peak_mem_gb = peak_mem / (1024**3)

        print(f"Epoch {epoch+1}/{num_epochs} | Batch {batch_size} | "
              f"TrainAcc={avg_train_acc:.4f} ValAcc={avg_val_acc:.4f} | "
              f"Time={epoch_time:.2f}s Mem={peak_mem_gb:.2f}GB")

        # Save Checkpoint (Overwrite current epoch)
        torch.save({
            'epoch': epoch+1,
            'model_state': model.state_dict(),
            'optimizer_state': optimizer.state_dict(),
            'scaler_state': scaler.state_dict()
        }, ckpt_path)

        # Log to CSV
        row = {
            "mode": "fsdp_amp",
            "gpu_count": 1,
            "batch_size": batch_size,
            "epoch": epoch + 1,
            "train_loss": avg_train_loss,
            "train_acc": avg_train_acc,
            "val_loss": avg_val_loss,
            "val_acc": avg_val_acc,
            "epoch_time": epoch_time,
            "peak_mem_bytes": peak_mem,
            "peak_mem_gb": peak_mem_gb
        }
        
        file_exists = os.path.exists(csv_path)
        pd.DataFrame([row]).to_csv(csv_path, mode="a", header=not file_exists, index=False)

    cleanup_dist()

# ============================================================
# 4. MAIN
# ============================================================

if __name__ == "__main__":
    mp.set_start_method("spawn", force=True)
    
    CSV_FILE = "FSDP_AMP_metrics.csv"
    
    # 1. Load Data
    print("Loading Dataset...")
    df = pd.read_csv("oral_cancer_balanced.csv")
    
    path_map = {}
    for root, dirs, files in os.walk("Data"):
        if "val" in dirs: dirs.remove("val")
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                path_map[file] = os.path.join(root, file)
                
    # 2. Configurations
    BATCH_SIZES = [64, 128, 512]
    NUM_EPOCHS = 10
    GPU_COUNT = 1
    
    print(f"Starting Single GPU AMP Benchmark. Saving to {CSV_FILE}")
    
    for bs in BATCH_SIZES:
        print(f"\n=== Starting Run: Batch Size {bs} ===")
        try:
            mp.spawn(
                train_worker,
                args=(GPU_COUNT, df, path_map, bs, NUM_EPOCHS, CSV_FILE),
                nprocs=GPU_COUNT,
                join=True
            )
        except Exception as e:
            print(f"Error running batch size {bs}: {e}")
            
    print(f"\nAll runs completed. Check {CSV_FILE} for results.")

Overwriting single_gpu_amp_benchmark.py


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
!python single_gpu_amp_benchmark.py

Loading Dataset...
Starting Single GPU AMP Benchmark. Saving to FSDP_AMP_metrics.csv

=== Starting Run: Batch Size 64 ===
[RESUME] Found checkpoint for Batch 64. Loading...
/home/padala.r/MyProject/single_gpu_amp_benchmark.py:151: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of